## Soluções para os exercícios da aula anterior:


In [ ]:
(define (my-nil? x)
  (eqv? x '()))

In [ ]:
(my-nil? 2)

In [ ]:
(my-nil? '())

In [ ]:
(define (my-and x y) 
  (cond (x (cond (y #t) 
                 (else #f))) 
        (else #f))) 

In [ ]:
(my-and #t #f)

In [ ]:
(my-and 1 2)

In [ ]:
(define (my-not x)
  (cond (x #f)
        (else #t)))

In [ ]:
(my-not #f)

In [ ]:
(my-not #t)

In [ ]:
(define (my-append x y)
  (cond ((my-nil? x) y)
        (else (cons (car x) (my-append (cdr x) y)))))

In [ ]:
(my-append '(1 2) '(3 4))

In [ ]:
(my-append '(1 2) 3)

In [ ]:
(define (my-pairs x y) 
  (cond ((my-nil? x) '())
        ((my-nil? y) '())
        ((my-and (pair? x) (pair? y))
         (cons (list (car x) (car y))
               (my-pairs (cdr x) (cdr y))))))

In [ ]:
(my-pairs '(1 2 3) '(a b c))

In [ ]:
(my-pairs '(1 2 3) '(a b c d))

In [ ]:
(define (my-assoc x y)
  (cond ((eqv? (caar y) x) (cadar y))
        (else (my-assoc x (cdr y)))))

In [ ]:
(my-assoc 'c '((a 1) (b 2) (c 3)))

In [ ]:
(my-assoc 'd '((a 1) (b 2) (c 3)))

## Continuando...
No final da última aula, comentei que as funções que foram implementadas têm um significado especial. Agora vamos ver o porquê!

### Começaremos definindo a função `atom?`
`atom?` recebe como argumento:

* `x`: um valor qualquer.

e retorna:

* `#f` se `x` for uma lista **não** vazia e `#t` caso contrário.

#### Com base nas últimas aulas, como seria essa função?

In [ ]:
(define (atom? x)
  (not (pair? x)))

In [ ]:
(atom? 'x)

In [ ]:
(atom? '(x))

In [ ]:
(atom? '())

### Agora vamos definir a função `my-apply`
`my-apply` recebe como argumentos:

* `fn`: uma função qualquer;
* `l`: uma **lista**.

e retorna:

* O valor de `fn` aplicado aos elementos de `l`.

In [ ]:
(define (my-apply fn l)
  (cond ((atom? fn) 
         (cond ((eqv? fn 'car)   (caar l))
               ((eqv? fn 'cdr)   (cdar l))
               ((eqv? fn 'cons)  (cons (car l) (cadr l)))
               ((eqv? fn 'atom?) (atom? (car l)))
               ((eqv? fn 'eqv?)  (eqv? (car l) (cadr l)))
               (else ?)))
         ((eqv? (car fn) 'label) ?)
         ((eqv? (car fn) 'lambda) ?)))

### Encontramos três problemas.
Mas vamos por partes...

Como o programa:
```scheme
(my-apply 'f '(1 2 3))
```
funcionaria se `my-apply` não consegue identificar o simbolo `'f`?

### A solução:
Precisamos adicionar um contexto para a função `my-apply`, onde ela armazenará novas definições.

Para isso, introduziremos um novo argumento `a` (de lista de `associações`) que será uma lista de associações (`assoc list` ou `alist` para os intimos).

### O que é uma `alist`?
Foi o `hashmap` antes do `hash` e `map` existir!!!

Ela é bem simples, é uma lista de listas no formato `((k1 v1) (k2 v2) ... (kn vn))`.

### Hmm... isso lembra algo! Certo?
Sim! A função `my-pairs` retorna uma `alist` e, melhor ainda, a função `my-assoc` busca o termo `x` na `alist y`!!!

### Redefinindo `my-apply`
`my-apply` recebe como argumentos:

* `fn`: uma função qualquer;
* `l`: uma **lista**;
* `a`: uma `alist` com o contexto atual. 

e retorna:

* O valor de `fn` aplicado aos elementos de `l` no contexto `a`, ou seja, substituindo todo simbolo de `l` com o seu correspondente em `a`.
  
  Casos especial: `define` adiciona um elementa a lista `a`. Exemplo: `(define x 2)` adiciona a associação `(x 2)` a `a` ao invés de buscar por `x` em `a`! 

In [ ]:
(define (my-apply fn l a)
  (cond ((atom? fn) 
         (cond ((eqv? fn 'car)   (caar l))
               ((eqv? fn 'cdr)   (cdar l))
               ((eqv? fn 'cons)  (cons (car l) (cadr l)))
               ((eqv? fn 'atom?) (atom? (car l)))
               ((eqv? fn 'eqv?)  (eqv? (car l) (cadr l)))
               (else ?)))
        ((eqv? (car fn) 'label) 
         (my-apply (caddr fn) l (cons (list (cadr fn) (caddr fn)) a))) 
        ((eqv? (car fn) 'lambda) ?)))

### Uma breve explicação:
Temos uma função capaz de receber outra função `fn`, uma lista `l = '(x1 x2 ... xn)` e um contexto `a` e calcular o valor de `(fn x1 x2 ... xn)`.

Mas isso só funciona para os casos que definimos:

* `car`: extrai o primeiro elemento de uma lista;
* `cdr`: extrai tudo de uma lista exceto o primeiro elemento;
* `cons`: cria uma lista com seu primeiro argumento no `inicio` (`car`) e seu segundo argumento como resto (`cdr`);
* `atom?`: devolve `#f` se seu argumento for uma lista **não** vazia e `#t` caso contrário;
* `eqv?`: retorna `#t` se ambos seus argumentos tiverem o mesmo valor e `#f` caso contrário;
* `label`: define uma nova função com `nome` igual seu primeiro argumento e `valor` igual seu segundo argumento (uma expressão `lambda`). Útil para recursões!

### Mas e agora?
Perceba que a função `lambda` precisa entender seu `corpo` para ser aplicada, ou seja, também não conseguimos aplicar ela sem antes conhecermos como computar qualquer **sexp**.

Como vimos na aula de cálculo λ e LISP, tudo em LISP são expressões S ou **sexp**s.

Enquanto `my-apply` lida com funções e seus argumentos, se faz necessário uma função que lida com qualquer **sexp**.

### Vamos definir a função `my-eval`
`my-eval` recebe como argumentos:
* `e`: uma **sexp** qualquer;
* `a`: uma `alist` com o contexto atual.

E retorna:
* o valor da expressão `e` no contexto `a`.

### Como assim????
Calma! Juro que tudo fará sentido em poucos minutos... (tic tac)

In [ ]:
(define (my-eval e a)
  (cond ((atom? e) (my-assoc e a))
        ((atom? (car e))
         (cond ((eqv? (car e) 'quote) (cadr e))
               ((eqv? (car e) 'cond)  ?)     ;; Precisaremos criar uma função auxiliar para lidar com o cond ...
               (else (my-apply (car e) ?)))) ;; Precisaremos de criar outra função auxiliar para lidar com contextos complexos ...
        (else (my-apply (car e) ?))))

### Como lidar com o cond?
Precisamos criar uma função que avalia cada predicado `p` por vez e, assim que algo difente `p` for diferente de `#f`,
deve retornar o a expressão associada a `p`. 

In [ ]:
(define (my-evcon c a)
    (cond ((my-eval (caar c) a)
           (my-eval (cadar c) a))
          (else (my-evcon (cdr c) a)))) 

##### Perceba: essa função percorre uma `alist` computando o valor de `k`, ou `(caar c)`, e, caso `k` seja diferente de `#f`, ela retorna `v`, ou `(cadar c)`. 

### Como lidar com contextos complexos?
Precisamos criar uma função capaz de percorrer uma `alist` e extrair o contexto de uma `sexp` agnósticamente, ou seja, independente da `sexp`. 

In [ ]:
(define (my-evlis m a)
  (cond ((my-nil? m) '())
        (else (cons (my-eval (car m) a)
                    (my-evlis (cdr m) a)))))

##### Perceba que o que acabamos de fazer é, "simplesmente", percorrer uma **árvore** (`sexp`), independente de seu formato! 
Computando toda `sexp` interna a original (recursivamente), até que encontremos uma lista vazia `'()`. 

### Certo! Voltemos ao `my-eval`

In [ ]:
(define (my-eval e a)
  (cond ((atom? e) (my-assoc e a))
        ((atom? (car e))
         (cond ((eqv? (car e) 'quote) (cadr e))
               ((eqv? (car e) 'cond)  (my-evcon (cdr e) a))
               (else                  (my-apply (car e) (my-evlis (cdr e) a) a))))
        (else (my-apply (car e) (my-evlis (cdr e) a) a))))

### Lembra que o `my-apply` está incompleto? Vamos termina-lo!

In [ ]:
(define (my-apply fn l a)
  (cond ((atom? fn) 
         (cond ((eqv? fn 'car)   (caar l))
               ((eqv? fn 'cdr)   (cdar l))
               ((eqv? fn 'cons)  (cons (car l) (cadr l)))
               ((eqv? fn 'atom?) (atom? (car l)))
               ((eqv? fn 'eqv?)  (eqv? (car l) (cadr l)))
               (else             (my-apply (my-eval fn a) l a))))
        ((eqv? (car fn) 'label) 
         (my-apply (caddr fn) l (cons (list (cadr fn) (caddr fn)) a))) 
        ((eqv? (car fn) 'lambda) 
         (my-eval (caddr fn) (my-append (my-pairs (cadr fn) l) a)))))

### A Surpresa
Podemos agora implementar a verdadeira SURPRESA!!!

Um interpretador LISP primitivo escrito em... LISP.

Para isso, criaremos uma função que une tudo o que acabamos de construir. 

Ela é bem simples, apenas uma linha.

Conseguem imaginar como?

In [ ]:
(define (my-evalquote fn l)
  (my-apply fn l '()))

### Juntando tudo em um só bloco!

In [ ]:
(define (my-apply fn l a)
  (cond ((atom? fn) 
         (cond ((eqv? fn 'car)   (caar l))
               ((eqv? fn 'cdr)   (cdar l))
               ((eqv? fn 'cons)  (cons (car l) (cadr l)))
               ((eqv? fn 'atom?) (atom? (car l)))
               ((eqv? fn 'eqv?)  (eqv? (car l) (cadr l)))
               (else             (my-apply (my-eval fn a) l a))))
        ((eqv? (car fn) 'label) 
         (my-apply (caddr fn) l (cons (list (cadr fn) (caddr fn)) a)))
        ((eqv? (car fn) 'lambda) 
         (my-eval (caddr fn) (my-append (my-pairs (cadr fn) l) a)))))

(define (my-eval e a)
  (cond ((atom? e) (my-assoc e a))
        ((atom? (car e))
         (cond ((eqv? (car e) 'quote) (cadr e))
               ((eqv? (car e) 'cond)  (my-evcon (cdr e) a))
               (else                  (my-apply (car e) (my-evlis (cdr e) a) a))))
        (else (my-apply (car e) (my-evlis (cdr e) a) a))))

(define (my-evlis m a)
  (cond ((my-nil? m) '())
        (else (cons (my-eval (car m) a)
                    (my-evlis (cdr m) a)))))

(define (my-evcon c a)
    (cond ((my-eval (caar c) a)
           (my-eval (cadar c) a))
          (else (my-evcon (cdr c) a)))) 

(define (my-evalquote fn l)
  (my-apply fn l '()))

### Como funciona?
Vejam os exemplos a seguir:

In [ ]:
(my-evalquote 'eqv? '(a a))

In [ ]:
(my-evalquote 'cons '(a (b c)))

In [ ]:
(my-evalquote '(lambda (x y) 
                 (cond ((eqv? x y) '#t)
                       ('#t        '#f))) 
              '(1 2))

In [ ]:
(my-evalquote '(label search 
                      (lambda (x y)
                        (cond ((eqv? y '())     '#f)
                              ((eqv? x (car y)) '#t)
                              ('#t (search x (cdr y)))))) 
              '(5 (1 2 3 4 5)))

In [ ]:
(my-evalquote '(label search 
                      (lambda (x y)
                        (cond ((eqv? y '())     '#f)
                              ((eqv? x (car y)) '#t)
                              ('#t (search x (cdr y)))))) 
              '(6 (1 2 3 4 5)))

### Também podemos manipular o ambiente:

In [ ]:
(my-eval 'x '((x 2) (y b)))

In [ ]:
(my-eval '(cond ((atom? x)  'atom)
                (não-existo 'pair))
         '((x 2) (y (1 2))))

In [ ]:
(my-eval '(f '(b c))
          '((f (lambda (x) (cons 'a x)))))

In [ ]:
(my-eval '((lambda (x) (cons 'a x)) '(b c))
         '((f (lambda (x) (cons 'a x)))))

In [ ]:
(my-eval '((label firstatom (lambda (x)
                               (cond ((atom? x) x)
                                     (else (firstatom (car x))))))
           y)
         '((y ((a b) (c d))) (else #t)))